In [19]:
import pandas as pd

In [20]:
game = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/rounds.csv')
table = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/table.csv')

In [21]:

game['Date'] = pd.to_datetime(game['Date'])

In [ ]:
matchdim = game[['MatchID', 'Date', 'MatchStatus', 'HomeTeamName', 'AwayTeamName']].drop_duplicates().reset_index(drop=True)
matchdim.set_index('MatchID')


In [ ]:
hometeamdim = game[['HomeTeamName', 'HomeScoreFT']].drop_duplicates().reset_index(drop=True)
hometeamdim.set_index('HomeTeamName')

In [ ]:
awayteamdim = game[['AwayTeamName', 'AwayScoreFT']].drop_duplicates().reset_index(drop=True)
awayteamdim.set_index('AwayTeamName')

In [ ]:
teamdim = table[['TeamID', 'TeamName', 'NameCode', 'TeamColors']].drop_duplicates().reset_index(drop=True)
teamdim.set_index('TeamID')

In [ ]:
standingsdim = table[['NameCode','Position','Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA']].drop_duplicates().reset_index(drop=True)
standingsdim.set_index('NameCode')

In [ ]:
facttable = game.merge(matchdim, on='MatchID')\
                .merge(teamdim, on='TeamID')\
                [['Round']]
facttable